In [4]:
!pip install nashpy

  Created wheel for nashpy: filename=nashpy-0.0.21-py3-none-any.whl size=15281 sha256=360014312fed7dd66c162f8e9199ae51286a7602b4ead138b962c5029cba2bb6
  Stored in directory: /root/.cache/pip/wheels/02/08/62/cf4fa931e0a317d180936b266169a57f4bb4eb801465bbe8a1
Successfully built nashpy


In [5]:
import nashpy as nash
import numpy as np

Definimos las matrices de recompensas:


In [6]:
A = np.array([[-8, 0], [-10, -1]]) # A es el jugador filas.
B = np.array([[-8, -10], [0, -1]]) # B es el jugador columnas.
juego1 = nash.Game(A,B)
juego1

Bi matrix game with payoff matrices:

Row player:
[[ -8   0]
 [-10  -1]]

Column player:
[[ -8 -10]
 [  0  -1]]

Encontramos las estrategias de Equilibrio:


In [7]:
equilibrios = juego1.support_enumeration()
for eq in equilibrios:
    print(eq)

(array([1., 0.]), array([1., 0.]))


De manera similar, para un juego mas complejo con estrategias dominantes:

In [8]:
A2 = np.array([[6, 4, 4, 3],
               [5, 6, 0, 5],
               [5, 3, 6, 4],
               [2, 2, 3, 6]]) # A es el jugador filas.
               
B2 = np.array([[3, 4, 1, 0],
               [4, 3, 2, 1],
               [0, 2, 1, 4],
               [0, 3, 3, 1]]) # B es el jugador columnas.
juego2 = nash.Game(A2, B2)
juego2

Bi matrix game with payoff matrices:

Row player:
[[6 4 4 3]
 [5 6 0 5]
 [5 3 6 4]
 [2 2 3 6]]

Column player:
[[3 4 1 0]
 [4 3 2 1]
 [0 2 1 4]
 [0 3 3 1]]

Las estrategias resultantes son aquellas que no son dominadas:

In [9]:
equilibrios = juego2.support_enumeration()
for eq in equilibrios:
    print(eq)

(array([0.5, 0.5, 0. , 0. ]), array([0.66666667, 0.33333333, 0.        , 0.        ]))


#Minimax para 3 en Raya

In [10]:

import time
import math

class Juego:
    def __init__(self):
        self.iniciar_juego()

    def iniciar_juego(self):
        self.estado_actual = [['.','.','.'],
                              ['.','.','.'],
                              ['.','.','.']]

        # X siempre juega primero:
        self.turno_jugador = 'X'

    def dibujar_tablero(self):
        for i in range(0, 3):
            for j in range(0, 3):
                print('{}|'.format(self.estado_actual[i][j]), end=" ")
            print()
        print()

    def es_valido(self, px, py):
        if px < 0 or px > 2 or py < 0 or py > 2:
            return False
        elif self.estado_actual[px][py] != '.':
            return False
        else:
            return True


    #Definir cuando acaba el juego
    def es_fin(self):
        # # Victoria en vertical:
        for i in range(0, 3):
            if (self.estado_actual[0][i] != '.' and
                self.estado_actual[0][i] == self.estado_actual[1][i] and
                self.estado_actual[1][i] == self.estado_actual[2][i]):
                return self.estado_actual[0][i]

        # Victoria en Horizontal:
        for i in range(0, 3):
            if (self.estado_actual[i] == ['X', 'X', 'X']):
                return 'X'
            elif (self.estado_actual[i] == ['O', 'O', 'O']):
                return 'O'

        # Victoria diagonal principal:
        if (self.estado_actual[0][0] != '.' and
            self.estado_actual[0][0] == self.estado_actual[1][1] and
            self.estado_actual[0][0] == self.estado_actual[2][2]):
            return self.estado_actual[0][0]

        # Victoria diagonal secundaria:
        if (self.estado_actual[0][2] != '.' and
            self.estado_actual[0][2] == self.estado_actual[1][1] and
            self.estado_actual[0][2] == self.estado_actual[2][0]):
            return self.estado_actual[0][2]

        # Mesa llena:
        for i in range(0, 3):
            for j in range(0, 3):
                # Si hay espacio, seguimos jugando:
                if (self.estado_actual[i][j] == '.'):
                    return None

        # Empate, mesa llena sin victoria:
        return '.'


        # 'O' es max, en este caso la IA:
    def max(self):

        # Posibles valores para max:
        # -1 - pierde
        #  0 - empata
        #  1 - gana

        # Inicializamos el resultado a -2, peor que lo peor
        maxv = -2

        px = None
        py = None

        resultado = self.es_fin()

        # Si el juego acaba, debemos devolver el resultado:
        if resultado == 'X':
            return (-1, 0, 0)
        elif resultado == 'O':
            return (1, 0, 0)
        elif resultado == '.':
            return (0, 0, 0)

        for i in range(0, 3):
            for j in range(0, 3):
                if self.estado_actual[i][j] == '.':
                    # Esto es una rama del juego:
                    self.estado_actual[i][j] = 'O'
                    (m, min_i, min_j) = self.min()
                    # Fijamos maxv si es necesario:
                    if m > maxv:
                        maxv = m
                        px = i
                        py = j
                    # Vaciamos el tablero
                    self.estado_actual[i][j] = '.'
        return (maxv, px, py)

    # El jugador 'X' es min, un humano en este caso:
    def min(self):

        #Inicio igual que para max:
        minv = 2

        qx = None
        qy = None

        resultado = self.es_fin()

        if resultado == 'X':
            return (-1, 0, 0)
        elif resultado == 'O':
            return (1, 0, 0)
        elif resultado == '.':
            return (0, 0, 0)

        for i in range(0, 3):
            for j in range(0, 3):
                if self.estado_actual[i][j] == '.':
                    self.estado_actual[i][j] = 'X'
                    (m, max_i, max_j) = self.max()
                    if m < minv:
                        minv = m
                        qx = i
                        qy = j
                    self.estado_actual[i][j] = '.'

        return (minv, qx, qy)
    def max_alfa_beta(self, alfa, beta):
            maxv = -2
            px = None
            py = None

            result = self.es_fin()

            if result == 'X':
                return (-1, 0, 0)
            elif result == 'O':
                return (1, 0, 0)
            elif result == '.':
                return (0, 0, 0)

            for i in range(0, 3):
                for j in range(0, 3):
                    if self.estado_actual[i][j] == '.':
                        self.estado_actual[i][j] = 'O'
                        (m, min_i, in_j) = self.min_alfa_beta(alfa, beta)
                        if m > maxv:
                            maxv = m
                            px = i
                            py = j
                        self.estado_actual[i][j] = '.'

                        if maxv >= beta:
                            return (maxv, px, py)

                        if maxv > alfa:
                            alfa = maxv

            return (maxv, px, py)



    def min_alfa_beta(self, alfa, beta):

        minv = 2

        qx = None
        qy = None

        result = self.es_fin()

        if result == 'X':
            return (-1, 0, 0)
        elif result == 'O':
            return (1, 0, 0)
        elif result == '.':
            return (0, 0, 0)

        for i in range(0, 3):
            for j in range(0, 3):
                if self.estado_actual[i][j] == '.':
                    self.estado_actual[i][j] = 'X'
                    (m, max_i, max_j) = self.max_alfa_beta(alfa, beta)
                    if m < minv:
                        minv = m
                        qx = i
                        qy = j
                    self.estado_actual[i][j] = '.'

                    if minv <= alfa:
                        return (minv, qx, qy)

                    if minv < beta:
                        beta = minv

        return (minv, qx, qy)


    def jugar(self):
        while True:
            self.dibujar_tablero()
            self.result = self.es_fin()

            if self.result != None:
                if self.result == 'X':
                    print('El ganador es X!')
                elif self.result == 'O':
                    print('El ganador es O!')
                elif self.result == '.':
                    print("Empate!")


                self.iniciar_juego()
                return
            # Turno del jugador
            if self.turno_jugador == 'X':

                while True:
                    start = time.time()
                    #(m, qx, qy) = self.min()
                    (m, qx, qy) = self.min_alfa_beta(-2, 2)
                    end = time.time()
                    print('Evaluation time: {}s'.format(round(end - start, 7)))
                    print('Recommended move: X = {}, Y = {}'.format(qx, qy))

                    px = int(input('X coordenada: '))
                    py = int(input('Y coordenada: '))

                    qx = px
                    qy = py

                    if self.es_valido(px, py):
                        self.estado_actual[px][py] = 'X'
                        self.turno_jugador = 'O'
                        break
                    else:
                        print('Movimiento no valido, prueba otra vez.')
            # Turno de la IA
            else:
                (m, px, py) = self.max_alfa_beta(-2, 2)
                #(m, px, py) = self.max()
                self.estado_actual[px][py] = 'O'
                self.turno_jugador = 'X'

In [ ]:
g = Juego()
g.jugar()

.| .| .| 
.| .| .| 
.| .| .| 

Evaluation time: 0.1131279s
Recommended move: X = 0, Y = 0
X coordenada: 0
Y coordenada: 0
X| .| .| 
.| .| .| 
.| .| .| 

X| .| .| 
.| O| .| 
.| .| .| 

Evaluation time: 0.0046394s
Recommended move: X = 0, Y = 1
X coordenada: 0
Y coordenada: 1
X| X| .| 
.| O| .| 
.| .| .| 

X| X| O| 
.| O| .| 
.| .| .| 

Evaluation time: 0.0003841s
Recommended move: X = 2, Y = 0
X coordenada: 2
Y coordenada: 0
X| X| O| 
.| O| .| 
X| .| .| 

X| X| O| 
O| O| .| 
X| .| .| 

Evaluation time: 8.49e-05s
Recommended move: X = 1, Y = 2
X coordenada: 1
Y coordenada: 2
X| X| O| 
O| O| X| 
X| .| .| 

X| X| O| 
O| O| X| 
X| O| .| 

Evaluation time: 1.81e-05s
Recommended move: X = 2, Y = 2
X coordenada: 2
Y coordenada: 2
X| X| O| 
O| O| X| 
X| O| X| 

Empate!


#Ejercicio: ¿Puedes modificar las funciones min y max para podar las ramas innecesarias? ¿Que le pasa al tiempo de procesamiento de la busqueda de la IA?

Modifica las funciones min, max y jugar con los ejemplos siguientes rellenando los espacios:

In [11]:
  def max_alfa_beta(self, alfa, beta):
          maxv = -2
          px = None
          py = None

          result = self.es_fin()

          if result == 'X':
              return (-1, 0, 0)
          elif result == 'O':
              return (1, 0, 0)
          elif result == '.':
              return (0, 0, 0)

          for i in range(0, 3):
              for j in range(0, 3):
                  if self.estado_actual[i][j] == '.':
                      self.estado_actual[i][j] = 'O'
                      (m, min_i, in_j) = self.min_alfa_beta(alfa, beta)
                      if m > maxv:
                          maxv = m
                          px = i
                          py = j
                      self.estado_actual[i][j] = '.'

                      if maxv >= beta:
                          return (maxv, px, py)

                      if maxv > alfa:
                          alfa = maxv

          return (maxv, px, py)   

In [12]:
  def min_alfa_beta(self, alfa, beta):

      minv = 2

      qx = None
      qy = None

      result = self.es_fin()

      if result == 'X':
          return (-1, 0, 0)
      elif result == 'O':
          return (1, 0, 0)
      elif result == '.':
          return (0, 0, 0)

      for i in range(0, 3):
          for j in range(0, 3):
              if self.estado_actual[i][j] == '.':
                  self.estado_actual[i][j] = 'X'
                  (m, max_i, max_j) = self.max_alfa_beta(alfa, beta)
                  if m < minv:
                      minv = m
                      qx = i
                      qy = j
                  self.estado_actual[i][j] = '.'

                  if minv <= alfa:
                      return (minv, qx, qy)

                  if minv < beta:
                      beta = minv

      return (minv, qx, qy)


##Recorrer todas las ramas

#### Evaluacion: 2.8568301s
#### Evaluacion: 0.0368521s
#### Evaluacion: 0.0012145s

##Utilizando alfa y beta

#### Evaluacion: 0.1131279s
#### Evaluacion: 0.0046394s
#### Evaluacion: 0.0003841s

#### Como resultado obtenemos un menor tiempo de ejecución y un mejor rendimiento. En este caso quizás no se pueda apreciar una gran diferencia debido a que es un ejercicio con pocos escenarios. Sin embargo, en problemas como el ajedrez se obtendrían resultados más relevantes.